<a href="https://colab.research.google.com/github/Cyanjiner/classroom-analysis/blob/main/transcript_EDA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Colab workspace
from google.colab import drive
drive.mount('/content/drive')

# installations for VMs
# !pip install transformers
# # !pip install -U sentence-transformers
# !pip install sentencepiece
!pip install textacy
import math, sys, os, requests, datetime, json
import numpy as np
import pandas as pd
## Text structures
import textblob
import nltk
import spacy
spacy.prefer_gpu()
# spacy.require_gpu()
# !python -m spacy download en_core_web_trf
import textacy
import textacy.text_stats
import textacy.preprocessing
from typing import Literal
from bs4 import BeautifulSoup


ROOT_FOLDER = "/content/drive/My Drive/transcript_analysis/"
DATA_FOLDER = ROOT_FOLDER + "data/"
RAW_FOLDER = DATA_FOLDER + "raw/"
CLEAN_FOLDER = DATA_FOLDER + "clean/"
MODELS_FOLDER = ROOT_FOLDER + "models/"
LOG_DIR = MODELS_FOLDER + "runs/"

In [ ]:
df = pd.read_csv(DATA_FOLDER + "ncte_single_utterances.csv")

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 580408 entries, 0 to 580407
Data columns (total 9 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   speaker       580395 non-null  object
 1   text          580408 non-null  object
 2   year          580408 non-null  int64 
 3   OBSID         580408 non-null  int64 
 4   video_id      580408 non-null  int64 
 5   cleaned_text  580358 non-null  object
 6   num_words     580408 non-null  int64 
 7   turn_idx      580408 non-null  int64 
 8   comb_idx      580408 non-null  object
dtypes: int64(5), object(4)
memory usage: 39.9+ MB


In [ ]:
pd.crosstab(df.)

,speaker,text,year,OBSID,video_id,cleaned_text,num_words,turn_idx,comb_idx
0,teacher,"Friends, yesterday we started off by working o...",2,2119,706,friends yesterday we started off by working on...,13,0,2119_0
1,student,Yes.,2,2119,706,yes,1,1,2119_1
2,teacher,"And yesterday towards the end of the period, y...",2,2119,706,and yesterday towards the end of the period yo...,38,2,2119_2
3,multiple students,Yes.,2,2119,706,yes,1,3,2119_3
4,teacher,Some of you might be done. If you are finishe...,2,2119,706,some of you might be done if you are finished ...,106,4,2119_4


In [ ]:
df.groupby(by=["OBSID"]).sum()

,year,video_id,num_words,turn_idx
OBSID,,,,
3,250,82250,4925,31125
4,465,152055,8156,107880
5,591,192666,6849,174345
7,523,178343,5859,136503
8,468,163800,6674,109278
...,...,...,...,...
4758,381,54356,3013,8001
4759,1233,220707,3055,84255
4760,2997,440559,8266,498501


In [ ]:
df.groupby(by=["OBSID","video_id"]).size()

OBSID  video_id
3      329          250
4      327          465
5      326          591
7      341          523
8      350          468
                   ... 
4758   428          127
4759   537          411
4760   441          999
4762   444          390
4763   832         1158
Length: 1661, dtype: int64

In [ ]:
df.video_id.value_counts()

729     4554
784     4485
419     4456
760     4376
443     4240
        ... 
513      284
539      282
548      279
1411     279
557      109
Name: video_id, Length: 319, dtype: int64

In [ ]:
df.speaker.value_counts()

teacher              286561
student              255268
multiple students     38566
Name: speaker, dtype: int64

In [ ]:
asdf = df[df.video_id == 706]
asdf

,speaker,text,year,OBSID,video_id,cleaned_text,num_words,turn_idx,comb_idx
0,teacher,"Friends, yesterday we started off by working o...",2,2119,706,friends yesterday we started off by working on...,13,0,2119_0
1,student,Yes.,2,2119,706,yes,1,1,2119_1
2,teacher,"And yesterday towards the end of the period, y...",2,2119,706,and yesterday towards the end of the period yo...,38,2,2119_2
3,multiple students,Yes.,2,2119,706,yes,1,3,2119_3
4,teacher,Some of you might be done. If you are finishe...,2,2119,706,some of you might be done if you are finished ...,106,4,2119_4
...,...,...,...,...,...,...,...,...,...
389930,teacher,How was pi created? We're actually going to d...,1,266,706,how was pi created we re actually going to dis...,16,170,266_170
389931,student,What if we run out of yarn?,1,266,706,what if we run out of yarn,7,171,266_171
389932,teacher,"I'll give you some more in a minute. Yes, Stu...",1,266,706,i ll give you some more in a minute yes student k,12,172,266_172
389933,student,Who made it up?,1,266,706,who made it up,4,173,266_173


In [ ]:
asdf.OBSID.value_counts()

626     375
2119    323
2639    306
2044    304
266     175
4039    145
760      94
4130     87
4717     84
Name: OBSID, dtype: int64

In [ ]:
ddf = df[df.OBSID == 2119]
ddf

,speaker,text,year,OBSID,video_id,cleaned_text,num_words,turn_idx,comb_idx
0,teacher,"Friends, yesterday we started off by working o...",2,2119,706,friends yesterday we started off by working on...,13,0,2119_0
1,student,Yes.,2,2119,706,yes,1,1,2119_1
2,teacher,"And yesterday towards the end of the period, y...",2,2119,706,and yesterday towards the end of the period yo...,38,2,2119_2
3,multiple students,Yes.,2,2119,706,yes,1,3,2119_3
4,teacher,Some of you might be done. If you are finishe...,2,2119,706,some of you might be done if you are finished ...,106,4,2119_4
...,...,...,...,...,...,...,...,...,...
318,teacher,Are what?,2,2119,706,are what,2,318,2119_318
319,student,Are not.,2,2119,706,are not,2,319,2119_319
320,teacher,Are not wearing ties. How many – so you don't...,2,2119,706,are not wearing ties how many so you don t wan...,171,320,2119_320
321,student,Where's the pictures?,2,2119,706,where s the pictures,4,321,2119_321


In [ ]:
df.columns

Index(['speaker', 'text', 'year', 'OBSID', 'video_id', 'cleaned_text',
       'num_words', 'turn_idx', 'comb_idx'],
      dtype='object')

In [ ]:
test = pd.read_csv('https://docs.google.com/spreadsheets/d/' + 
                   '0Ak1ecr7i0wotdGJmTURJRnZLYlV3M2daNTRubTdwTXc' +
                   '/export?gid=0&format=csv',
                   # Set first column as rownames in data frame
                   index_col=0,
                   # Parse column values to datetime
                   parse_dates=['Quradate']
                  )
test.head(5)  # Same result as @TomAugspurger

In [ ]:
## Getting exemplary language
url = f"https://docs.google.com/spreadsheets/d/13e28Y7l4sAQ5p-TEkRIb5MJ-T58Mba5ZMp7PaBBjWKQ/edit#gid=1086911083"
url_1 = url.replace('/edit#gid=', '/export?format=csv&gid=')
# url = f"https://docs.google.com/spreadsheets/d/13e28Y7l4sAQ5p-TEkRIb5MJ-T58Mba5ZMp7PaBBjWKQ/edit#gid=1086911083"
ldf = pd.read_csv(url_1)

In [ ]:
url = ldf[ldf.Course == "Grade 5"].iloc[0]["IM Demo hyperlink"]
r1 = requests.get(url)
page = r1.content
page_data = BeautifulSoup(page, 'html5lib')

In [ ]:
print(page_data.prettify())

<!DOCTYPE html>
<html lang="en">
 <head>
  <meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
  <script type="text/javascript">
   window.NREUM||(NREUM={});NREUM.info={"beacon":"bam.nr-data.net","errorBeacon":"bam.nr-data.net","licenseKey":"NRJS-899242e8384bdf15d3a","applicationID":"1374961817","transactionName":"dVcITEsNXg1TFEpBEV9VB0pATUYEVwUNVBFFFwpdShFdD0VJCVQQRVcI","queueTime":4,"applicationTime":524,"agent":""}
  </script>
  <script type="text/javascript">
   (window.NREUM||(NREUM={})).init={ajax:{deny_list:["bam.nr-data.net"]}};(window.NREUM||(NREUM={})).loader_config={licenseKey:"NRJS-899242e8384bdf15d3a",applicationID:"1374961817"};;(()=>{var __webpack_modules__={507:(__unused_webpack_module,__webpack_exports__,__webpack_require__)=>{"use strict";function detectPolyfillFeatures(){const featureStatus={};return checkAndAddFeature("Promise","PROMISE"),checkAndAddFeature("Array.prototype.includes","ARRAY_INCLUDES"),checkAndAddFeature("Object.assign","OBJECT_ASSIG

In [ ]:

tb = page_data.find_all('div')
tb = page_data.find_all(id="im-c-content")
len(tb)

0

In [ ]:
t = page_data.find('main').get_text()
t


"\nLesson 1\nWhat Is Volume? \n\nPreparationLesson\n\nWarm-up: Which One Doesn’t Belong: Objects Made of Cubes (10 minutes) \n\nCCSS Standards\nBuilding Towards\n5.MD.C.3Routines and Access\nInstructional Routines\nWhich One Doesn’t Belong?\n\n\n\n\nNarrative\n\nThe purpose of this warm-up is for students to compare four objects made of cubes and reason about their configuration. Responses related to the orientation of the object and the number of cubes in each will be applicable to the upcoming activities.\n\nFor all warm-up routines, consider establishing a small, discreet hand signal that students can display to indicate they have an answer they can support with reasoning. This signal could be a thumbs-up, a certain number of fingers that tells the number of responses they have, or a different subtle signal. This is a quick way to see if students have had enough time to think about the problem. It also keeps students from being distracted or rushed by hands being raised around the c